# 分散ブロックチェーンの実装

- 目標は分散ブロックチェーンで各ノードが、ドローンの離着陸地点の予約を管理してるポイントだとして、ドローンの着陸離陸許可がされるさまをつくる事

- [Bitcoin論文](https://bitcoin.org/bitcoin.pdf)をもとにblockchainの実装をする
    - 天下り的にならないよう，徐々にシステムをパワーアップさせていく方針で作る
    - 目標はシンプルなコンセンサスアルゴリズムの実装まで行い，ノード同士が競ってマイニングする様子を眺めること
    - 完成形はこんな感じ
    
    <img src="./demo.gif" width=50%>

# 信用のある第三者を介する取引
- まずは第三者を介した取引をシミュレーションしてみる
- 第三者は取引の公正さのすべてを請け負う

## 取引はどのように行うか? - Transactionの実装
- 取引記録を発行することで，取引を行う
    - Python3.7から実装された[dataclasses](https://docs.python.org/ja/3/library/dataclasses.html#)を使ってみる

In [1]:
from __future__ import annotations
from dataclasses import dataclass

In [2]:
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
import binascii

In [3]:
import sys
sys.path.append('src/')

- 個人の識別を`address`で行う
- 取引額を`value`とする

In [4]:
@dataclass
class Transaction:
    sender_address: int
    receiver_address: int
    value: float 

- senderはTransactionを発行して取引を行う

- Aliceがtransactionを発行してBankに登録することで，取引が成立する

- the_leader_nodeがトランザクションを発行して…等、解釈の変更が必要 

- ここで例えば，第三者の信頼度がちょっと落ちたとする
    - 取引内容の改ざんはどのように防げばよいだろうか?

# 電子署名を用いたtransactionの発行
- senderはtransactionに電子署名を行う
    - 今回はRSAを用いることにする
        - BitCoinは[ECDSA](https://en.bitcoin.it/wiki/Transaction#Input)を用いている
    - これによってtransactionの改ざんを検知することができる
- 電子署名の公開鍵はどのように扱う?
    - 公開鍵をWalletのaddressとする!

## Transactionの改良 - 電子署名フィールドの追加
- 電子署名できるようにする
    - `sign`を追加
    - 署名の際，transactionをstrで表すための`str_data` methodも追加
- ついでにデータをjson stringに変換するmethodを追加する
    - これは後々データのやり取りを簡単にするため

In [5]:
import json
import dataclasses as dc

In [6]:
@dataclass
class Transaction:
    sender_address: str
    receiver_address: str
    value: float
    sign: str = None
        
    def str_data(self) -> str:
        # return the str of data without `sign`
        d=dc.asdict(self)
        del d["sign"]
        return json.dumps(d)
    
    def json_dumps(self) -> str:
        return json.dumps(dc.asdict(self))
    @classmethod
    def json_loads(cls, string) -> Transaction:
        return cls(**json.loads(string))

## Walletの改良 - 電子署名機能の追加
- RSAによる電子署名機能を実装する
    - pycryptoを用いる
        - 使い方はこれを参照 ([python \- Signing and verifying data using pycrypto \(RSA\) \- Stack Overflow](https://stackoverflow.com/questions/4232389/signing-and-verifying-data-using-pycrypto-rsa))
- 引き続きWalletはaddressを1つのみ持つことにする
    - 例えばBitCoinでは匿名性の向上のため，一つのWalletは複数のキーペアを持つことができる ([参考](https://en.bitcoin.it/wiki/Transaction))

In [7]:
# from Crypto.Signature import PKCS1_v1_5
# from Crypto.Hash import SHA
# from Crypto.PublicKey import RSA
# import binascii

- pycryptoのkey instanceをstrに変換するhelper functionを実装する
    - strにしておく理由は後ほどわかる

In [8]:
def decode_key(key):
    if isinstance(key, str):
        return key
    return binascii.hexlify(key.exportKey(format='DER')).decode('ascii')
def encode_key(key):
    if isinstance(key, RSA.RsaKey):
        return key
    return RSA.importKey(binascii.unhexlify(key))

Walletの改良

- 秘密鍵`private_key`の追加 
- addressは対応する公開鍵とする
- `sign_transaction`の追加
    - transactionを受け取り，電子署名付きtransactionを返す
- `send`を変更
    - 電子署名付きtransactionを生成できるようになった!

つぎに，transactionの電子署名をチェックするhelper functionを作る
- signatureのpublic keyはtransactionのaddressなので，transactionを見れば電子署名の正当性を判別できる

In [9]:
def verify_transaction(transaction) -> bool:
    if transaction.sign is None:
        return False
    # hash the transaction
    h=SHA.new(transaction.str_data().encode())
    # generate verifier with public key
    verifier=PKCS1_v1_5.new(encode_key(transaction.sender_address))
    # is the signature correct?
    return verifier.verify(h, binascii.unhexlify(transaction.sign))

## デモ: RSAを用いたtransactionの発行とチェック
先ほどと同じようにデモをしてみる

- transactionを発行し，電子署名を行い，bankに提出する

- transactionの電子署名を確認してみる

- データをちょと変えてみると，不正なtransactionになるのがわかる

- これは後にできると面白いかもしれない



- transactionの改竄リスクを低減することができた 
    - しかし依然として，**double-spending problem** を防ぐには信頼のある第三者が必要となる
    - これはデータで通貨を作る際の最も大きな障害だった(データは簡単にコピーできるから)
- すべてのtransactionが公開されていれば，double-spendingは防げるのでは？
    - 公開されていればreceiverも記録をチェックできる

# Timestamp Server によるTransactionの公開
- いくつかのtransactionをまとめて，timestampをつけ，**block**で公開することを考える
    - 信頼のある**timestamp server**がtransactionをまとめてtimestampをつけ，電子署名をして公開する
    - ある特定の時刻にそのtransactionが存在したことが保証される
- さらに，blockには直前のtimestampのhashを含め，**blockchain**を作る
    - blockを改ざんすると，以降のblockは全て不正なものとなる
- これでtransactionをチェックしてdouble-spendingを防ぐ第三者は必要なくなった
    - transactionは公開されているから，誰でも検証することができる

## Blockの実装
- strで表現できるよう，json methodを追加する
- ついでにhash methodも実装する

In [10]:
from typing import Tuple, Sequence
import hashlib

In [11]:
from block import Block

## Timestamp Serverの実装
- timestamp serverは以下の流れで**blockchain**を作る
    1. transactionのリストを受け取る
    2. 一つ前のblockのhashとtimestampをつけてBlockにまとめる
    3. **電子署名**をする
    4. 公開する

- blockの正当性の保証はtimestamp serverの**電子署名**で行う
    - serverに RSA キーペアをもたせる
- 最初のblockはprevious blockがないため，特別なブロック(**genesis**)を用意する ([参考](https://en.bitcoin.it/wiki/Genesis_block))
- blockchainは今回はただのリストにする

In [12]:
from time import time

In [13]:
# class TimestampServer:
#     def __init__(self):
#         key=RSA.generate(1024)
#         self.public_key=key.publickey()
#         self.signer=PKCS1_v1_5.new(key)
        
#         genesis=Block(time(), (), "0")
#         self.block_chain=[genesis]
        
#     def generate_block(self, transactions: Sequence[Transaction]):
#         # generate block
#         block=Block(time(), tuple(transactions), self.block_chain[-1].hash())
        
#         # sign the block
#         dct=dc.asdict(block)
#         del dct["sign"]
#         block.sign=self.signer.sign(SHA.new(json.dumps(dct).encode())).hex()
        
#         # publish the block
#         self.block_chain.append(block)

## blockchainの正当性確認
- blockchainの正当性を確認するhelper functionを用意する
    1. blockの持つ**previous_hash**は正しいか?
    2. blockに入っているtransactionは正当か?
    3. timestamp serverがつけた電子署名は正当か?

- blockの正当性確認

In [14]:
# def verify_block(previous_block, block, timestamp_server_publickey) -> bool:
#     is_correct_hash = previous_block.hash() ==  block.previous_hash
#     is_correct_transactions = all(map(verify_transaction, block.transactions))
    
#     dct=dc.asdict(block)
#     del dct["sign"]
#     h=SHA.new(json.dumps(dct).encode())
#     verifier=PKCS1_v1_5.new(timestamp_server_publickey)
#     is_correct_sign=verifier.verify(h, binascii.unhexlify(block.sign))
#     return is_correct_hash and is_correct_transactions and is_correct_sign

- blockchainの正当性確認

In [15]:
# def verify_blockchain(chain, timestamp_server_publickey):
#     for i in range(len(chain)-1):
#         if not verify_block(chain[-i-2],chain[-i-1],timestamp_server_publickey):
#             return False
#     return True

# Proof-of-Workの実装
- これがblockchainの革新的要素
- 先程は"信頼の置ける" timestamp serverの電子署名が，Blockの正当性を保証していた
- では誰でも同じ署名ができれば，誰でもtimestamp serverの仕事ができるのでは?
    - 電子署名でやると，当然ブロックの正当性の証明にはならない..
- **[HashCash](https://en.bitcoin.it/wiki/Hashcash)**を用いる

## Blockにnonceを加える
- これがtimestamp serverの電子署名の代わりとなる
- blockのハッシュ値が適当な条件を満たすように**nonce**を追加する
    - 例えば上4桁が0
    - この制約を**difficulty**と呼ぶ
    - difficultyは適当に調整される. 例えばbitcoinの場合は，blockの追加のintervalが10分程度になるように調整されるようだ
        - [Difficulty \- Bitcoin Wiki](https://en.bitcoin.it/wiki/Difficulty)
    - 今回はdifficultyは固定にする
- nonceを見つける作業が**mine**
    - nonceを見つけるのは難しい(たくさん試して，あたりを引くしかない)
    - これによってblockの生成を非常に高コストにすることができる
    - nonceが正しいことを確認するのは簡単
        - 電子署名と同様, 正当性の確認が簡単に行える
- **mine**は誰でも行える

- Blockにnonceをつける．今回の変更はこれだけ

In [16]:
@dataclass
class Block:
    time: float
    transactions: Tuple[Transaction]
    previous_hash: str
    nonce: int = None
        
    def json_dumps(self) -> str:
        dct=dc.asdict(self)
        dct["transactions"]=[t.json_dumps() for t in self.transactions]
        return json.dumps(dct)
    
    @classmethod
    def json_loads(cls, string) -> Block:
        dct=json.loads(string)
        dct["transactions"]=tuple([Transaction.json_loads(t) for t in dct["transactions"]])
        return cls(**dct)
        
    def hash(self): 
        block_bytes=self.json_dumps().encode()
        return hashlib.sha256(block_bytes).hexdigest()

## nonce の正当性確認functionを実装

In [17]:
difficulty=3
def valid_proof(block):
    return block.hash()[:difficulty] == "0" * difficulty

## mineの実装
- あたりを引くまで手当たり次第試す

In [18]:
def mine(block):
    nonce=0
    block.nonce=nonce
    while not valid_proof(block):
        nonce += 1
        block.nonce=nonce
    return block

- ついでにverify_blockも改良

In [19]:
def verify_block(previous_block, block) -> bool:
    is_correct_hash = previous_block.hash() ==  block.previous_hash
    is_correct_transactions = all(map(verify_transaction, block.transactions))
    is_correct_proof = valid_proof(block)
    return is_correct_hash and is_correct_transactions and is_correct_proof

- BlockChainもちょっと改良
    - 自身をstrに変換するmethodをつける

In [20]:
from blockchain import BlockChain

# class BlockChain(list):
#     def json_dumps(self) -> str:
#         return json.dumps([block.json_dumps() for block in self])
#     @classmethod
#     def json_loads(cls, string) -> BlockChain:
#         return cls([Block.json_loads(s) for s in json.loads(string)])

## デモ: tranasctionの発行からminingまで

# ＊＊＊＊＊NodeとConsensus Algorithm
- 前回までで分散管理の準備は整った
- 具体的にblockchainをつくる**node**を実装する
- nodeやsender, receiverが集まって**network**を形成する
- networkの動きは以下のようになる

1. senderがtransaction生成．なるべく多くのnodeにbroadcastする（全てのnodeにbroadcastする事を確認する）
2. nodeはtransactionを集めてblockにまとめる
3. mine（必要ない）
4. 他のnodeにマイニングしたblockをbroadcastする
5. blockを受け取ったnodeは，blockの正当性を確認し，double-spendingのチェックをする（チェックはいくらしても良い、必要に応じて外して行く）
6. チェックが通れば，次のblockの作成を始める（チェックはないのでいつでもblockを作成できるが、ブロックチェーンではないのか）

- Nodeが複数いて，異なるBlockChainを持つときは，どのように正しさを判断するか？
    - **Consensus Algorithm**
    - 最も長いblockchainを，最も正しいblockchainとみなす
        - なぜなら，chainが長ければ長いほど，多くの計算資源が注ぎ込まれたことになるから
        - 多数決ではなく，**one-CPU-one-vote**

- Nodeの役割を考え直す必要がある

- Nodeが複数いて、異なる分散台帳を持つが、それらは論理的に単一である
    - **Consensus Algorithm** は簡単なものであり、許可型の分散台帳において、論理的な単一性を保持する

In [21]:
from uuid import uuid4
from time import sleep
from copy import deepcopy
import multiprocessing as mp
import random
from itertools import count
import node 

- nodeを実装する
- networkまわりの細かい作業は，network interfaceに任せる

- 改めて、いわゆるマイニングを必要とするノードにおいては、ノードが複雑であると思われる。
- コンフリクトは起こらない（２つのコンセンサスアルゴリズムを用意して、その様子を比較検討できるようにする）

In [22]:
class Node:
    def __init__(self, network, genesis, uuid=None):
        self.chain=BlockChain([genesis])
        self.uuid=uuid or str(uuid4())
        self.network=network
        
    def work(self, verbose=True):
        while True:
            if not self.ordering():
                sleep(1.0) # wait due to no transactions
            if self.add_block():
                if verbose: print(self.uuid, "Added one block")
            self.network.post_chain(self.chain, self.uuid)        
            if self.resolve_conflicts():
                if verbose: print(self.uuid, "Change chain")

    def ordering(self):
        transactions=self.network.get_transactions(self.uuid)
        if len(transactions)==0:
            return False # cannot mine due to no transactions
        previous_hash=self.chain[-1].hash()
        block=Block(time(), tuple(transactions), previous_hash)
                
        if self.add_block(): 
            return True
        if self.resolve_conflicts():
            return True
                
        self.network.broadcast_block(block, self.uuid)
        return True # successfully mined

    def add_block(self):
        for block in self.network.get_blocks(self.uuid):
            # if self.verify_block(block):
                self.chain.append(block)           
                if self.verify_chain(self.chain):
                    return True
                else:
                    self.chain.pop(-1)

        return False # no block is added

    def resolve_conflicts(self):
        """Longest valid chain is authoritative"""
        # authoritative_chain=self.chain
        authoritative_chain=self.chain  
        for chain in self.network.get_neighbour_chains(self.uuid):
            if not self.verify_chain(chain):
                # node is incorrect
                continue
            if len(chain)>len(authoritative_chain):
                # Longest valid chain is authoritative
                authoritative_chain=deepcopy(chain)
        self.chain=authoritative_chain 
        # self.chain=deepcopy(self.network.get_chain('the_leader_node')) 
        return self.chain is not authoritative_chain
        
    @staticmethod
    def verify_transaction(transaction):
        if transaction.sign is None:
            return False
        h=SHA.new(transaction.str_data().encode())
        verifier=PKCS1_v1_5.new(encode_key(transaction.sender_address))
        return verifier.verify(h, binascii.unhexlify(transaction.sign))        

    def verify_block(self, block) -> bool:
        is_correct_transactions = all(map(self.verify_transaction, block.transactions))
        return is_correct_transactions
    
    def verify_chain(self, chain):
        for i in range(len(chain)-1, 0, -1):
            # if not self.verify_block(chain[i]):
            #     return False
            if chain[i-1].hash() != chain[i].previous_hash:
                return False
        return True

- 全体的に，networkの作業が追加されている
- mineを少し変更している
    - 複数のnodeが同じ方法でマイニングをするのは無駄が生じる可能性がある
        - 例えば全く同じBlockのマイニングを複数のノードが開始した場合，確実に最も計算力の高いnodeがマイニングを成功させることになる

- Walletもネットワーク対応に
    - transactionを発行したら，知っているnodeにtransactionを送る

- networkっぽいものを実装する
    - Network classを中継点としてデータの遣り取りをする
- あとでmultiprocessでノードを動かすために，データの管理はmultiprocess.managerで行う
- flaskとかで作っても面白いかもしれない
- 本質的でないので読み飛ばして良い

In [23]:
from typing import List

In [24]:
class Network:
    def __init__(self, neighbours=()):
        self.manager=mp.Manager()
        self.chains=self.manager.dict()
        self.blocks=self.manager.dict()
        self.transactions=self.manager.dict()
        self.neighbours=dict(neighbours)
        
    def post_chain(self, chain, uuid):
        self.chains[uuid]=chain.json_dumps()
    
    def get_leader_chain(self, uuid) -> BlockChain:
        if uuid not in self.chains:
            return []
        return self.chains[uuid]       

    def get_chain(self, uuid) -> BlockChain:
        if uuid not in self.chains:
            return []
        return BlockChain.json_loads(self.chains[uuid])
    
    def get_neighbour_chains(self, uuid) -> List[BlockChain]:
        return [self.get_chain(neigh) for neigh in self.neighbours[uuid] if neigh != uuid]
    
    def post_block(self, block, uuid):
        if uuid not in self.blocks:
            self.blocks[uuid]=self.manager.list([block.json_dumps()])
        else:
            self.blocks[uuid].append(block.json_dumps())
            
    def broadcast_block(self, block, uuid):
        for receiver in self.neighbours[uuid]:
            self.post_block(block, receiver)
            
    def get_blocks(self, uuid) -> List[Block]:
        if uuid not in self.blocks:
            return []
        res=[Block.json_loads(s) for s in self.blocks[uuid]]
        self.blocks[uuid][:]=[]
        return res
    
    def post_transaction(self, transaction, uuid):
        if uuid not in self.transactions:
            self.transactions[uuid]=self.manager.list([transaction.json_dumps()])
        else:
            self.transactions[uuid].append(transaction.json_dumps())
            
    def get_transactions(self, uuid) -> Tuple[Transaction]:
        if uuid not in self.transactions:
            return []
        res=[Transaction.json_loads(s) for s in self.transactions[uuid]]
        self.transactions[uuid][:]=[]
        return res
    
    def shutdown(self):
        self.manager.shutdown()

## ノード1つの場合のテスト

In [25]:
genesis=Block(time(), (), "0")
uuid='the_leader_node'
network=Network({uuid:[uuid]})
node=Node(network, genesis, uuid)

### トランザクションはライブラリ化される必要がある

In [26]:
import json
import dataclasses as dc

In [27]:
@dataclass
class Transaction:
    sender_address: int
    receiver_address: int
    value: float 

### 今回可読性を重要視したため、暗号化は試用していない

In [28]:
# from Crypto.Signature import PKCS1_v1_5
# from Crypto.Hash import SHA
# from Crypto.PublicKey import RSA
# import binascii

In [29]:
from wallet import Wallet

In [30]:
alice=Wallet(network, [uuid])
bob=Wallet(network, [uuid])

- これを実行してしばらく問題が起きなければOK
    - 適当にinterruptする

In [31]:
# node.work(verbose=True)


- 後片付け

In [32]:
network.shutdown()


# デモ: 分散ブロックチェーンのデモ
- multiprocessingでnetworkのデモを行う

In [33]:
genesis=Block(time(), (), "0")
network=Network()

- ノードを生成する

- uuid はテストで分かりやすいように名前を与えてある

In [34]:
# num_nodes=2
# nodes=[str(uuid4()) for _ in range(num_nodes-1)]
# nodes=['leader'] + nodes
nodes=['the_leader_node', 'the_blue_sky', 'white_noise', 'yellow_submarine', 'green_planet']
# nodes=['the_leader_node', 'the_blue_sky']


- nodeのnetworkの状態を定義する
- とりあえずは全結合にする
    - つまり全てのNode同士，直接やりとりできる

- 1つのnodeはただ１つの決まった結合を持つ場合、コンフリクトは起こらない
- ブロックが作成されるたびにリーダーにブロードキャストしたい

- ネットワーク・トポロジーを可視化したい

In [35]:
network.neighbours['the_leader_node']=['the_leader_node', 'the_blue_sky', 'white_noise', 'yellow_submarine', 'green_planet']
network.neighbours['the_blue_sky']=['the_leader_node', 'the_blue_sky', 'the_leader_node']
network.neighbours['white_noise']=['the_leader_node', 'white_noise', 'the_leader_node']
network.neighbours['yellow_submarine']=['the_leader_node', 'yellow_submarine', 'the_leader_node']
network.neighbours['green_planet']=['the_leader_node', 'green_planet', 'the_leader_node']

In [36]:
# for uuid in nodes:
#     network.neighbours[uuid]=nodes

- multiprocess workerを作って起動する

In [37]:
def node_work(uuid):
    node=Node(network, genesis, uuid)
    node.work()

In [38]:
workers=[]
for uuid in nodes:
    worker=mp.Process(target=node_work, args=(uuid,))
    workers.append(worker)
    worker.start()

- peopleを生成する
- これも適当に人数を変えて遊ぶと良い

- peopleは概念としては今回はあまり試用していない

In [39]:
# num_people=2
# people=[Wallet(network, random.sample(nodes, random.randint(1, len(nodes)))) for _ in range(num_people)]
alice=Wallet(network, ['the_leader_node'])
bob=Wallet(network, ['the_blue_sky'])
yamamoto=Wallet(network, ['the_white_noise'])

people=[alice, bob, yamamoto]

### blockchain の表示
- 各ノードの持つblockchainがどの様になっているのか表示する
- 別Processで行う
    - Jupyterは賢いのでoutputにきちんと表示してくれる

In [40]:
from IPython.display import clear_output
import time

timeout = time.time() + 60
def draw_chains(network):
    while True:
        if time.time() > timeout:
            break
        line=""
        for k, v in network.chains.items():
            line += f"(Node: {k[:10]}) "
            chain=BlockChain.json_loads(v)
            h=list(map(lambda x: x.hash()[:5], chain))
            line += "--".join(map(lambda x: f"[{x}]", h))
            line += "\n"
        print(line.rstrip())
        clear_output(True)
        sleep(1.0)


worker=mp.Process(target=draw_chains, args=(network,))
worker.start()

(Node: the_leader) [ee8e3]
(Node: the_blue_s) [ee8e3]
(Node: white_nois) [ee8e3]
(Node: yellow_sub) [ee8e3]
(Node: green_plan) [ee8e3]


- 取引をして遊ぶ
    - ランダムに取引が行われるコードを書いてみる
    - 上の表示が変化する!

In [41]:
def generate_transactions():
    # for _ in range(random.randint(1,10)):
    #     sender, receiver= random.sample(people, 2)
    #     # sender.send(receiver.address, random.randint(1, 100))
    #     sender.send(receiver.address, 'inquiry')

    alice.send([None], 'cancel')
    bob.send([None], 'reservation')

In [42]:
generate_transactions()

NameError: name 'Transaction' is not defined

### スマートコントラクトの実装に向けて

In [ ]:
# import Smartcontract

In [ ]:
# import json
# import network
# import wallet
import collections

class Smartcontract:

    def __init__(self, network):
        self.state = {"sender_address": None, "receiver_address": None, "reserved": 0}
        self.network = network

    #トランザクションの履歴を取得する
    def get_history_of_transactions_by_uuid(self, uuid):
        history = []
        for chain in json.loads(self.network.get_chain(uuid).json_dumps()):
                transactions = json.loads(chain).get('transactions')
                for t in transactions:
                    history.append(t)  

        return history             

    def get_current_state(self, uuid='the_leader_node'):
        for operand in self.get_history_of_transactions_by_uuid(uuid):
            d_operand = json.loads(operand) #convert to a dictionary
            if d_operand['value']=='reservation':
                if not self.state["reserved"]:
                    self.state["sender_address"] = d_operand["sender_address"]
                    self.state["receiver_address"] = d_operand["receiver_address"]
                    self.state["reserved"] = 1
            elif d_operand['value']=='cancel':
                if self.state["reserved"]:
                    if ((self.state["sender_address"]==d_operand["sender_address"])):
                            self.state["sender_address"] = None
                            self.state["receiver_address"] = None
                            self.state["reserved"] = 0
        return self.state

    def cancel_reservation(self, s):
        self.state = self.get_current_state('the_leader_node')
        # print(self.state['sender_address'][0])
        # print(s.address[0])        
        if (self.state['reserved'] and (collections.Counter(self.state['sender_address']) == collections.Counter(s.address))):
            # alice.send(bob.address, 'cancel')      
            s.send([None], 'cancel')   
            return True
        return False

    def make_a_reservation(self, s):
        self.state = self.get_current_state('the_leader_node')
        if not self.state['reserved']:
            # alice.send(bob.address, 'reservation')     
            s.send([None], 'reservation')         
            return True
        return False

### メッセージを出力した方が良いかもしれない

In [ ]:
sc = Smartcontract(network)

s_uuid = 'the_leader_node'
s = Wallet(network, [s_uuid])
b_uuid = 'the_blue_sky'
b = Wallet(network, [b_uuid])

In [ ]:
clear_output(True)

print(s_uuid + ' は離着陸地点を予約しました。' if sc.make_a_reservation(s) else s_uuid + ' は離着陸地点は予約できませんでした。')

print('以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('以下ブロックチェーンの源蔵のステート')
print(sc.get_current_state(s_uuid))

NameError: name 'Transaction' is not defined

In [ ]:
clear_output(True)

print(s_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(s) else s_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

print('以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

print('以下ブロックチェーンの源蔵のステート')
sc.get_current_state(s_uuid)

NameError: name 'sc' is not defined

In [ ]:
print(b_uuid + ' は離着陸地点を予約しました。' if sc.make_a_reservation(b) else b_uuid + ' は離着陸地点は予約できませんでした。')

print('以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

sc.get_current_state(s_uuid)

NameError: name 'sc' is not defined

In [ ]:
print(b_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(b) else b_uuid + ' によっては離着陸地点はキャンセルできませんでした。')
# print(s_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(s) else s_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

print('以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

sc.get_current_state(s_uuid)

NameError: name 'sc' is not defined

In [ ]:
print('以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)

sc.get_current_state(s_uuid)

以下ブロックチェーンの内容（トランザクションの履歴）


NameError: name 'sc' is not defined

In [ ]:
print(b_uuid + ' により離着陸地点の予約はキャンセルされました。' if sc.cancel_reservation(b) else b_uuid + ' によっては離着陸地点はキャンセルできませんでした。')

print('以下ブロックチェーンの内容（トランザクションの履歴）')
for index, each in enumerate(sc.get_history_of_transactions_by_uuid(s_uuid)):
    print(str(index) +' '+ each)


sc.get_current_state(s_uuid)

NameError: name 'sc' is not defined

### 片付け

In [ ]:
worker.terminate()
worker.kill()
for worker in workers:
    worker.terminate()
    worker.kill()
network.shutdown()

# おわりに
- 本物のBitCoinとは異なる部分がたくさんあります

# 参考文献
- [Bitcoin white paper](https://bitcoin.org/bitcoin.pdf)
    - 専門外でも読みやすいです
- [Bitcoin Wiki](https://en.bitcoin.it/wiki/Main_Page)
    - 非常に情報量が多いです